In [1]:
import csv
import sys
import pandas as pd

In [2]:
# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}

small_direct = 'degrees/degrees/small'

large_direct = 'degrees/degrees/large'

def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

In [3]:
def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]

In [4]:
class Node():
    def __init__(self, state, parent, movie):
        self.state = state
        self.parent = parent
        self.movie = movie

In [5]:
class QueueFrontier():
    
    def __init__(self):
        self.frontier = []
        
    def add(self, node):
        self.frontier.append(node)
        
    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)
    
    def empty(self):
        return len(self.frontier) == 0
        
    def remove(self):
        if self.empty():
            raise Exception('empty frontier')
        else:
            node = self.frontier[0] # first in, first out
            self.frontier = self.frontier[1:]
            return node

In [6]:
class short_path():
    
    def __init__(self, source, target):
        self.source = source
        self.target = target
        
        
    def solve(self):
        source_id = self.source
        target_id = self.target
        
        start = Node(state = source_id, parent = None, movie = None)
        frontier = QueueFrontier()
        frontier.add(start)
        
        ids_explored = set()
        
        path_found = False
        
        while path_found is False:
            
            # If nothing left in frontier, then no path
            if frontier.empty():
                raise Exception('no solution')
                
            ''' Did the following after initializing the while loop so that it keeps changing after the for loop
            and the frontier has a list of neighbors, frontier will have a list of all the neighbors explored in the
            for loop'''
            
            node = frontier.remove()
            neighbor = neighbors_for_person(node.state)
            
            ids_explored.add(node.state)

            for stars in neighbor:
                movie_id = stars[0]
                person_id = stars[1]
                
                ''' Parent turned into node so that it is easy to back track the parent based on the current state'''
                
                new = Node(state = person_id, parent = node, movie = movie_id)

                if person_id == target_id:
                    print('Found Target')
                    movies = []
                    people = []

                    while new.parent is not None:
                        movies.append(new.movie)
                        people.append(new.state)
                        new = new.parent
                       
                    movies.append(None)
                    people.append(self.source)
                    
                    movies.reverse()
                    people.reverse()
                    
                    self.solution = movies, people
                    return self.solution
                    
                elif person_id not in ids_explored:
                    frontier.add(new) 
                    
                    ''' If the person ID does not match the target ID then do this so that the frontier
                    can store the list of IDs we will handle later on'''


In [7]:
def shortest_path(source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.

    If no possible path, returns None.
    
    """
    
    path1 = short_path(source, target)
    return path1.solve()

In [10]:
def main():

    directory = small_direct

    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("Data loaded.")

    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")

    path = shortest_path(source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path[0]) - 1
        
        print(f"{degrees} degrees of separation.")
        
        print(path)
        
        for i in range(degrees):
            person1 = people[path[1][i]]["name"]
            person2 = people[path[1][i + 1]]["name"]
            movie = movies[path[0][i + 1]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")

In [11]:
def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    
    for movie_id in movie_ids:
        for star_id in movies[movie_id]["stars"]:
            if star_id != person_id:
                neighbors.add((movie_id, star_id))
                
    return neighbors


if __name__ == "__main__":
    main()

Loading data...
Data loaded.
Name: Tom Hanks
Name: Sally Field
Found Target
1 degrees of separation.
([None, '109830'], ['158', '398'])
1: Tom Hanks and Sally Field starred in Forrest Gump


In [ ]:
shortest_path('Tom Hanks', 'Chris Saradon')

In [ ]:
people['158']['movies']